In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
import time
import csv

In [2]:
driver = webdriver.Chrome()
driver.get("https://www.noon.com/")
time.sleep(3)

In [3]:
page = 1
all_products = []

with open('Noon_AppleWatch_data.csv', mode='w', newline='', encoding='utf-8-sig') as file:
    writer = csv.writer(file)
    writer.writerow(['Product Name', 'Price', 'Link', 'Rating', 'Reviews'])

    while page <= 15: # ✅ LIMIT TO 15 PAGES        
        base_url = f"https://www.noon.com/uae-en/search/?q=Apple%20Watch%20Series&limit=50&page={page}&sort[by]=popularity&sort[dir]=desc"
        driver.get(base_url)
        time.sleep(5)
        # ✅ 
        products = driver.find_elements(By.XPATH, '//a[contains(@class, "ProductBoxLinkHandler_productBoxLink__FPhjp")]')
        print(f"\n Page {page} — Products found: {len(products)}")  

        if len(products) == 0:
            print("No more products found. Scraping complete.")
            break

        for product in products:
            try:
                product_name = product.find_element(By.XPATH, ".//h2[@data-qa='plp-product-box-name']").text
                try:
                    amount = product.find_element(By.XPATH, './/div[contains(@class, "Price_sellingPrice")]//strong').text.strip()
                    price = f"AED {amount}"
                except:
                    price = "N/A"


                Link = product.get_attribute("href")

                try:
                    Rating = product.find_element(By.XPATH, './/div[contains(@class, "RatingPreviewStar_textCtr")]').text
                except:
                    Rating = "N/A"
                try:
                    rating = product.find_element(By.XPATH, ".//div[contains(@class, 'RatingPreviewStar_countCtr')]//span").text
                except:
                    rating = "No Rating"
                try:
                    review_element = WebDriverWait(product, 5).until(
                        EC.presence_of_element_located(
                            (By.XPATH, './/div[contains(@class, "RatingPreviewStar_countCtr")]//span')
                        )
                    )
                    Reviews = review_element.text
                except TimeoutException:
                    Reviews = "N/A"           


                all_products.append([product_name, price, Link, Rating, Reviews])
                writer.writerow([product_name, price, Link, Rating, Reviews])

                print(f"Product: {product_name} | Price: AED {price} | Link: {Link} | Rating: {Rating} | Reviews: {Reviews}")

            except Exception as e:
                print(f"Error extracting product: {e}")

        page += 1  # ✅ Page increase loop ke andar hai

# ✅ Convert to DataFrame
df = pd.DataFrame(all_products, columns=["Product Name", "AED Price", "Link", "Rating", "Reviews"])
df.to_csv("Noon_AppleWatch_data.csv", index=False, encoding="utf-8-sig")
print("\n Data saved to mobiles_data.csv and loaded into pandas DataFrame.")



 Page 1 — Products found: 62
Product: Apple Watch Series 8 GPS 41mm Aluminium Case With Sport Band Midnight Sport Band | Price: AED AED 887 | Link: https://www.noon.com/uae-en/watch-series-8-gps-41mm-aluminium-case-with-sport-band-midnight-sport-band/N53349192A/p/?o=c24e8bc0c08819aa | Rating: 4.5 | Reviews: 6.6K
Product: Apple Watch Series 10 GPS 46mm Jet Black Aluminium Case With Black Sport Band | Price: AED AED 1,431 | Link: https://www.noon.com/uae-en/watch-series-10-gps-46mm-jet-black-aluminium-case-with-black-sport-band/N70108561V/p/?o=ac155545dc4aae4b | Rating: 4.7 | Reviews: 3.6K
Product: Apple Watch SE (2024) GPS 40mm Starlight Aluminium Case With Starlight Sport Band | Price: AED AED 725 | Link: https://www.noon.com/uae-en/watch-se-2024-gps-40mm-starlight-aluminium-case-with-starlight-sport-band/N70108651V/p/?o=cc5bb83bdd4c632d | Rating: 4.6 | Reviews: 1.8K
Product: Apple Watch SE (2024) GPS 40mm Midnight Aluminium Case With Midnight Sport Band | Price: AED AED 749 | Link: h